# Eigengene Single-Variant Association Preparation

**Created**: 24 March 2022

## Environment

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE)) {
    install.packages("BiocManager")
}

if (!requireNamespace("ComplexHeatmap", quietly=TRUE)) {
    BiocManager::install("ComplexHeatmap")
}

In [2]:
library(tidyverse)
library(RColorBrewer)
library(ComplexHeatmap)
library(data.table)

setwd("~/eQTL_pQTL_Characterization/")

source("04_Expression/scripts/utils/ggplot_theme.R")

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: grid

ComplexHeatmap version 2.6.2
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations

## Load Data

In [3]:
demographics <- readxl::read_xls("/nfs/team282/data/gains_team282/ClinicalData/DEMO_12jun2019.xls", sheet=1) %>%
    dplyr::mutate(sex=as.numeric(sex)) %>%
    dplyr::mutate(diagnosis=as.numeric(diagnosis))

ga.prefix.index <- grepl("GA", demographics$SubjectBarCode)
demographics$SubjectBarCode[ga.prefix.index] <- gsub("^GA", "", demographics$SubjectBarCode[ga.prefix.index])

In [4]:
srs.info <- read.table("/nfs/team282/data/gains_team282/full-gains-SRS-predictions_mNN-RF.tsv", header=T)
ga.prefix.index <- grepl("GA", srs.info$Sample_id)
srs.info$Sample_id[ga.prefix.index] <- gsub("^GA", "", srs.info$Sample_id[ga.prefix.index])
srs.info <- srs.info %>%
  dplyr::filter(Assay=="RNA-seq")
row.names(srs.info) <- srs.info$Sample_id

In [5]:
covs <- read.table("~/gains_team282/eqtl/data/covs_and_peer_factors.txt") %>%
    dplyr::mutate(Sample.ID=rownames(.)) %>%
    dplyr::mutate(GAinS.ID=sapply(strsplit(Sample.ID, "_"), function(x) { x[1] }))

ga.prefix.index <- grepl("GA", covs$GAinS.ID)
covs$GAinS.ID[ga.prefix.index] <- gsub("^GA", "", covs$GAinS.ID[ga.prefix.index])
covs$Sample.ID[ga.prefix.index] <- gsub("^GA", "", covs$Sample.ID[ga.prefix.index])

In [6]:
held.out <- c("Neutrophils", "Lymphocytes", "Monocytes", paste0("PC", 1:7), "Diagnosis", "SRSq", "sex")
peer <- paste0("PEER_", 1:30)

covs <- covs %>%
    merge(., demographics, by.x="GAinS.ID", by.y="SubjectBarCode") %>%
    merge(., srs.info, by.x="Sample.ID", by.y=0) %>%
    dplyr::select(diagnosis=diagnosis, everything()) %>%
    dplyr::select(Sample.ID, any_of(held.out), any_of(peer)) %>%
    as.data.frame()
                                  
rownames(covs) <- covs$Sample.ID
covs <- covs %>%
    dplyr::select(-Sample.ID)
                                  
head(covs)

,Neutrophils,Lymphocytes,Monocytes,PC1,PC2,PC3,PC4,PC5,PC6,PC7,⋯,PEER_21,PEER_22,PEER_23,PEER_24,PEER_25,PEER_26,PEER_27,PEER_28,PEER_29,PEER_30
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
UK01050129_5,-1.7381318,1.8281826,1.2829455,0.177279994,0.051694602,0.02486030,-0.035102502,-0.00455149,-0.0603707,0.03964880,⋯,-0.001172263,-0.05533835,0.049742281,0.03725176,0.04495762,0.12696229,0.08175036,0.05295999,0.164950684,-0.10379386
UK01060123_3,-0.3845370,0.3878375,0.4548023,-0.000388133,-0.000674099,0.00810357,0.000212791,0.03396150,0.0266317,-0.01885280,⋯,-0.012723563,0.02226736,-0.027520355,-0.05782387,0.03653939,0.05959492,-0.02611259,-0.01790827,0.012047987,0.05554044
UK01070117_3,-0.7738039,0.5846032,0.9325467,0.016432300,-0.048464298,0.00813966,-0.047699600,-0.07926830,0.0263629,-0.08224740,⋯,0.040162142,0.06840651,0.000062529,0.05292596,0.01705806,0.05931079,0.08962730,0.03772458,0.008254956,-0.01570964
UK01080111_1,0.1305646,0.2888638,-0.6232047,0.161789000,0.036224801,0.00467903,-0.048611499,0.00180036,-0.0352357,0.05012510,⋯,-0.002928361,-0.03274319,0.088269033,0.07396927,-0.01906336,0.02454052,0.04243371,0.03055887,0.123383760,-0.02276623
UK01110093_1,1.0909779,-1.2488160,-0.4429439,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983,⋯,-0.108193189,0.01590979,-0.093242317,-0.03037383,-0.07385907,-0.05968965,0.09695430,-0.03975484,0.130198911,-0.05011296
UK01110093_5,0.2302416,0.0000000,-0.3306479,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983,⋯,-0.018054215,0.02094322,-0.008623131,0.09560122,-0.07802416,0.02365689,0.08040638,-0.02349218,0.189585552,-0.07635343


In [7]:
eigengenes <- read.csv("~/gains_team282/nikhil/expression/gene_expression/eigengenes.clr.csv", row.names=1)

head(eigengenes)

,ME_1,ME_2,ME_3,ME_4,ME_5,ME_6,ME_7,ME_8,ME_9,ME_10,ME_11,ME_12,ME_13
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
UK02270173_3,-0.026880028,0.024754255,-0.049255630,-0.0011365576,-0.02004220,-0.011621161,-0.048982210,0.001510054,-0.03801025,0.001552109,0.002506806,-0.01010815,-0.018126419
UK15130120_3,-0.012686164,-0.012329614,-0.049890308,0.0117096753,-0.05134518,0.011795701,-0.032446820,-0.006916683,-0.01832666,0.022868622,0.023239704,-0.01033057,-0.001873539
UK58000006_3,-0.007422916,0.019682349,-0.003439649,-0.0004373064,0.01823638,-0.037003556,-0.026203330,0.013285317,-0.00654476,-0.034106149,-0.033193160,0.01994496,0.021248987
UK47010004_3,0.010860942,0.005580810,-0.001070395,-0.0123976557,0.01688644,-0.002396543,0.010436675,-0.009976259,-0.03845000,0.044978110,0.013607013,-0.04736191,-0.021150863
UK42020088_5,-0.020354540,0.013730057,-0.005233625,-0.0288076192,-0.01357106,0.034517070,-0.003360994,0.013865936,-0.03183148,-0.032342665,-0.018588091,0.02292120,0.003827274
UK47490007_3,0.007358504,-0.007576762,-0.004897662,-0.0114631104,-0.02199959,0.033353492,0.015936130,-0.046998162,-0.01985218,0.043605091,0.074601996,-0.02636748,-0.015488821


In [8]:
modules <- read.csv("~/gains_team282/nikhil/expression/gene_expression/modules.clr.csv")

In [9]:
head(modules)

,Gene,Module,Module.Pruned
,<chr>,<chr>,<chr>
1,ENSG00000002549,Unassigned,Unassigned
2,ENSG00000004534,Unassigned,Unassigned
3,ENSG00000004799,Unassigned,Unassigned
4,ENSG00000005379,Unassigned,Unassigned
5,ENSG00000006468,Unassigned,Unassigned
6,ENSG00000006757,Unassigned,Unassigned


In [10]:
gene.exp <- read.table("/lustre/scratch119/humgen/projects/gains_team282/eqtl/data/logcpm_864_20412_hla.txt")

In [11]:
head(gene.exp)

,UK02270173_3,UK15130120_3,UK58000006_3,UK47010004_3,UK42020088_5,UK47490007_3,UK02770164_3,UK02770164_5,UK02630151_3,UK42150107_1,⋯,UK59070043_3,UK59070043_5,UK02510223_3,UK02XX0336_5,UK29090086_3,UK02XX0335_1,UK02XX0334_3,UK01210130_3,UK01210130_5,UK01380125_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000230521,0.6644339,0.58271604,0.4046756,0.20905157,0.2211571,0.4288519,0.5133516,0.57768877,0.6990766,0.3812037,⋯,0.3823184,0.5423684,0.86762630,0.73833461,0.6461583,0.1227063,0.9621386,0.65838459,1.03561474,0.4457301
ENSG00000225864,0.9817872,0.43879936,0.6022344,0.92340489,0.2211571,1.1450686,0.1921362,0.57768877,0.1974851,1.2290495,⋯,0.6842527,0.8539227,0.91886777,0.12581620,0.5511865,0.4384007,0.5465139,0.40803066,0.46899151,0.3544115
ENSG00000227766,1.2117156,0.30682669,0.6625010,0.79963286,0.6445686,0.3130290,0.1183231,0.16750450,0.1505900,1.0533199,⋯,0.4901384,0.4340781,0.37504164,0.04316943,0.6151905,0.3406488,0.3147798,0.43511377,0.17382742,0.1157514
ENSG00000237669,0.6420763,0.69256830,0.9792012,0.69929584,0.3765156,1.0756817,0.8912050,0.60129641,0.8534668,1.0533199,⋯,1.0790156,0.6102946,1.03953331,0.97965846,0.7061683,0.7489857,0.7919934,0.85168813,0.69120049,1.0798771
ENSG00000271581,2.4744895,1.33153140,1.8310464,2.47531701,1.8812042,2.0531366,0.5972544,1.00686213,1.2811361,2.4244938,⋯,2.4728831,1.5338898,1.59080138,0.62923925,1.2804253,1.4727511,1.2187650,1.46471386,1.44139443,1.1896250
ENSG00000285647,0.0000000,0.03379072,0.0000000,0.05517067,0.6748270,0.0000000,0.0000000,0.03509293,0.6990766,1.6556266,⋯,3.4117627,2.4012963,0.04682116,0.00000000,0.0000000,0.4695662,0.3762952,0.07078527,0.07206309,0.4159282


In [12]:
lead.cis.eqtl <- read.table("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/eigenMT/ciseqtl_eigenMT_corrected.txt") %>%
    dplyr::filter(Sig)

In [13]:
head(lead.cis.eqtl)

,snps,gene,statistic,pvalue,beta,se,chr,SNPpos,TSS,BF,TESTS,BF.FDR,Sig,threshold
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<lgl>,<dbl>
22,rs3131972,ENSG00000237491,5.643405,2.400875e-08,0.07174216,0.012712570,1,817341,778747,7.058573e-06,294,1.993414e-05,TRUE,9.170798e-05
23,rs3131972,ENSG00000230092,4.756587,2.329109e-06,0.06239499,0.013117600,1,817341,817712,6.917454e-04,297,1.593483e-03,TRUE,9.078164e-05
25,rs3131972,ENSG00000225880,6.292961,5.949542e-10,0.10016770,0.015917420,1,817341,827522,1.767014e-07,297,5.668498e-07,TRUE,9.078164e-05
28,rs2272757,ENSG00000188976,-7.026045,4.904859e-12,-0.05107767,0.007269761,1,946247,959309,1.545031e-09,315,5.762103e-09,TRUE,8.559411e-05
29,rs13303327,ENSG00000187961,-7.175012,1.845538e-12,-0.13519920,0.018843060,1,960326,960584,5.813445e-10,315,2.228468e-09,TRUE,8.559411e-05
30,rs13303056,ENSG00000187583,-9.081031,1.922802e-18,-0.29633770,0.032632610,1,953778,966482,6.056826e-16,315,3.262796e-15,TRUE,8.559411e-05


In [14]:
conditional.cis.eqtl <- readRDS("/nfs/users/nfs_n/nm18/gains_team282/eqtl/cisresults/conditionalanalysis/conditional_eQTL_results_final.rds")

In [15]:
head(conditional.cis.eqtl)

,SNP,Gene,eQTL_beta,eQTL_SE,pvalue,Number
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,rs10753794,ENSG00000000457,0.0498587312408011,0.00708515851579373,5.19259471106013e-12,1
2,rs10919255,ENSG00000000460,-0.0726264894633498,0.0136853304567293,1.72815915933528e-07,1
3,rs77006036,ENSG00000000460,0.166236949102582,0.0334472538699365,8.47290238610159e-07,2
4,rs12406047,ENSG00000000971,0.17135374034767,0.0379807691167313,7.51523462853891e-06,1
5,rs6696136,ENSG00000001460,0.45884434698694,0.0871081626842823,1.94887370606405e-07,1
6,rs6676449,ENSG00000001460,-0.371531778822554,0.0880672792106829,2.8471651589813e-05,2


In [16]:
ebi.studies <- fread("04_Expression/data/gwas_catalog_v1.0.2-studies_r2022-02-21.tsv", header=TRUE, quote="") %>%
    as.data.frame()

In [17]:
head(ebi.studies, n=1)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,PLATFORM [SNPS PASSING QC],ASSOCIATION COUNT,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
,<date>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,2021-08-24,34124712,Sherva R,2021-02-28,Explor Med,www.ncbi.nlm.nih.gov/pubmed/34124712,Genome-wide association study of phenotypes measuring progression from first cocaine or opioid use to dependence reveals novel risk genes.,Cocaine dependence (time to event),"3,554 African American cases, 478 African American controls, 2,712 European ancestry cases, 915 European ancestry controls","572 African American cases, 416 African American controls, 759 European ancestry cases, 1,620 European ancestry controls",Illumina [NR] (imputed),5,cocaine dependence,http://www.ebi.ac.uk/efo/EFO_0002610,GCST012225,Genome-wide genotyping array


In [18]:
ebi.assoc <- fread("04_Expression/data/gwas_catalog_v1.0.2-associations_e105_r2022-02-21.tsv", header=TRUE, quote="") %>%
    as.data.frame()

In [19]:
head(ebi.assoc, n=1)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,⋯,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
,<date>,<int>,<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2021-07-07,34127860,Robertson CC,2021-06-14,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/34127860,"Fine-mapping, trans-ancestral and genomic analyses identify causal variants, cells, genes and drug targets for type 1 diabetes.",Type 1 diabetes,"20,065 European ancestry cases, 33,065 European ancestry controls, 1,045 other-admixed ancestry cases, 1,103 other-admixed ancestry controls, 1,043 African ancestry cases, 3,206 African ancestry controls",NA,⋯,8.522879,,0.912,,Illumina [715631] (imputed),N,type 1 diabetes mellitus,http://purl.obolibrary.org/obo/MONDO_0005147,GCST90013445,Targeted genotyping array [ImmunoChip]


In [20]:
geno.bim <- fread("~/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.bim") %>%
    as.data.frame()
colnames(geno.bim) <- c("chr", "snps", "cM", "pos", "minor", "major")

In [21]:
head(geno.bim)

,chr,snps,cM,pos,minor,major
,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,1,rs3131972,0,817341,A,G
2,1,rs546843995,0,818053,0,G
3,1,rs553916047,0,818359,0,A
4,1,1:818740_T_C,0,818740,T,C
5,1,rs145604921,0,819378,0,C
6,1,rs535256652,0,821053,0,T


## Gene Expression PCs

The module refinement procedure discards ~9000 genes. I will calculate PCs from these genes to control for technical factors not accounted for in other covariates. I am hypothesizing that the modules formed from the rest of the genes represent true biological signals. I'll include the first 30 PCs.

In [22]:
unassigned.genes <- modules$Gene[modules$Module.Pruned == "Unassigned"]
unassigned.gene.exp <- t(gene.exp[unassigned.genes,])
unassigned.gene.exp.svd <- svd(scale(unassigned.gene.exp))

In [23]:
gene.exp.pcs <- unassigned.gene.exp.svd$u[,1:30]
rownames(gene.exp.pcs) <- rownames(unassigned.gene.exp)
pcs <- paste0("Gene_Exp_PC", 1:30)
colnames(gene.exp.pcs) <- pcs
rownames(gene.exp.pcs) <- gsub("^GA", "", rownames(gene.exp.pcs))
gene.exp.pcs <- gene.exp.pcs[rownames(covs),]

In [24]:
head(gene.exp.pcs)

,Gene_Exp_PC1,Gene_Exp_PC2,Gene_Exp_PC3,Gene_Exp_PC4,Gene_Exp_PC5,Gene_Exp_PC6,Gene_Exp_PC7,Gene_Exp_PC8,Gene_Exp_PC9,Gene_Exp_PC10,⋯,Gene_Exp_PC21,Gene_Exp_PC22,Gene_Exp_PC23,Gene_Exp_PC24,Gene_Exp_PC25,Gene_Exp_PC26,Gene_Exp_PC27,Gene_Exp_PC28,Gene_Exp_PC29,Gene_Exp_PC30
UK01050129_5,-0.05945679,7.396396e-02,0.01274124,-0.0112844921,0.012751830,0.007860453,-0.008250262,0.046134299,0.003959458,0.014904068,⋯,0.002442630,0.014472714,0.010122804,-0.018568895,0.02955028,-0.018029125,-0.005774272,-0.033913856,-0.014132836,0.012273754
UK01060123_3,-0.01560333,2.349240e-02,-0.02082291,0.0448819271,0.009481692,0.028692445,0.018137045,-0.014255415,-0.017246112,-0.010657504,⋯,0.029781414,-0.009111470,-0.042289180,-0.018598499,0.03365637,-0.025156985,-0.020932081,-0.001319561,-0.007350224,0.025998600
UK01070117_3,-0.03104763,-5.886191e-05,-0.01057338,0.0008902746,0.009227810,0.023485116,-0.050541578,-0.004042473,-0.008954768,-0.034349355,⋯,-0.009692787,0.000137702,0.002470854,0.014315828,0.01444792,0.014043154,-0.026104068,-0.032637196,-0.029486023,0.014349461
UK01080111_1,0.00745554,1.941599e-02,0.01388253,0.0112125466,0.066831632,-0.059039021,0.002001893,-0.060396348,0.068444620,0.004825214,⋯,-0.008891282,0.003956666,0.043050247,0.015125900,0.01472175,-0.007187568,0.021612025,-0.038603337,0.013234403,0.001215820
UK01110093_1,0.02810750,-1.101635e-01,-0.05651708,-0.0284398381,0.034401915,0.026758751,0.054521354,-0.014044159,-0.002242296,-0.001428255,⋯,-0.023762273,0.061202049,-0.011192400,-0.001689001,-0.02657633,0.039272582,0.081409446,-0.020022599,-0.010265247,-0.004249738
UK01110093_5,-0.02173342,-1.091444e-01,-0.04905253,-0.0245717564,-0.039118389,0.017869684,0.038262004,0.058891249,0.012330741,0.074703597,⋯,-0.014952651,-0.019676711,-0.017587738,0.027756715,-0.00406966,0.034642964,0.034914416,-0.010948103,-0.034823690,0.004866058


In [25]:
covs <- cbind(covs, gene.exp.pcs)

In [26]:
head(covs)

,Neutrophils,Lymphocytes,Monocytes,PC1,PC2,PC3,PC4,PC5,PC6,PC7,⋯,Gene_Exp_PC21,Gene_Exp_PC22,Gene_Exp_PC23,Gene_Exp_PC24,Gene_Exp_PC25,Gene_Exp_PC26,Gene_Exp_PC27,Gene_Exp_PC28,Gene_Exp_PC29,Gene_Exp_PC30
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
UK01050129_5,-1.7381318,1.8281826,1.2829455,0.177279994,0.051694602,0.02486030,-0.035102502,-0.00455149,-0.0603707,0.03964880,⋯,0.002442630,0.014472714,0.010122804,-0.018568895,0.02955028,-0.018029125,-0.005774272,-0.033913856,-0.014132836,0.012273754
UK01060123_3,-0.3845370,0.3878375,0.4548023,-0.000388133,-0.000674099,0.00810357,0.000212791,0.03396150,0.0266317,-0.01885280,⋯,0.029781414,-0.009111470,-0.042289180,-0.018598499,0.03365637,-0.025156985,-0.020932081,-0.001319561,-0.007350224,0.025998600
UK01070117_3,-0.7738039,0.5846032,0.9325467,0.016432300,-0.048464298,0.00813966,-0.047699600,-0.07926830,0.0263629,-0.08224740,⋯,-0.009692787,0.000137702,0.002470854,0.014315828,0.01444792,0.014043154,-0.026104068,-0.032637196,-0.029486023,0.014349461
UK01080111_1,0.1305646,0.2888638,-0.6232047,0.161789000,0.036224801,0.00467903,-0.048611499,0.00180036,-0.0352357,0.05012510,⋯,-0.008891282,0.003956666,0.043050247,0.015125900,0.01472175,-0.007187568,0.021612025,-0.038603337,0.013234403,0.001215820
UK01110093_1,1.0909779,-1.2488160,-0.4429439,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983,⋯,-0.023762273,0.061202049,-0.011192400,-0.001689001,-0.02657633,0.039272582,0.081409446,-0.020022599,-0.010265247,-0.004249738
UK01110093_5,0.2302416,0.0000000,-0.3306479,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983,⋯,-0.014952651,-0.019676711,-0.017587738,0.027756715,-0.00406966,0.034642964,0.034914416,-0.010948103,-0.034823690,0.004866058


## Heatmap

Before performing single-variant association analysis, we want to check if any covariates are strongly associated with the module eigengenes.

In [27]:
all.vars <- merge(covs, eigengenes, by.x=0, by.y=0) %>%
    dplyr::select(-Row.names)

In [28]:
colors <- colorRampPalette(brewer.pal(11, "RdBu"))(101)

cor.mtx <- cor(all.vars)
cor.mtx.peer <- cor.mtx[peer, colnames(eigengenes)]
cor.mtx.pcs <- cor.mtx[pcs, colnames(eigengenes)]
cor.mtx.held.out <- cor.mtx[held.out, colnames(eigengenes)]

h1 <- Heatmap(cor.mtx.pcs, col=colors, heatmap_legend_param=list(
    title="Correlation",
    at=c(-1, 0, 1)
))

h2 <- Heatmap(cor.mtx.peer, col=colors, heatmap_legend_param=list(
    title="Correlation",
    at=c(-1, 0, 1)
))

h3 <- Heatmap(cor.mtx.held.out, col=colors, cluster_rows=F, heatmap_legend_param=list(
    title="Correlation",
    at=c(-1, 0, 1)
))

svg("04_Expression/results/eigengene_peer_factor_correlation.svg", width=8, height=12)
h1 %v% h2 %v% h3
dev.off()

png 
  2

The PEER factors are built using gene expression data. It's not surprising that many of the eigengenes are associated with the PEER factors. Module refinement discarded around 9000 genes. The residual gene expression variation from these genes is likely captured by some of the PEER factors. We model this specifically using 30 gene expression PCs from the discarded genes.

The eigengenes are not associated with genotyping PCs (which is good).

![](../results/eigengene_peer_factor_correlation.svg)

Ideally, I will include only sex and genotyping PCs as covariates in this mapping, since everything else (cell counts, SRS group) are important signatures that may be correlated with module eigengenes. I will include 20 PEER factors to control for technical confounders and to remain consistent with the cis-eQTL mapping procedure.

## Save Covariates

Save list of patients (family ID in the first column and individual ID in the second column) to subset the genotyping data using PLINK. There are 638 unique patients with genotyping information.

In [29]:
eigengene.patients <- sapply(strsplit(rownames(eigengenes), "_"), function(x) { x[1] })

geno.fam <- fread("~/gains_team282/Genotyping/All_genotyping_merged_filtered_b38_refiltered_rsID.fam") %>%
    dplyr::select(Family.ID=1, Individual.ID=2) %>%
    dplyr::mutate(GAinS.ID=gsub("^GA", "", Individual.ID)) %>%
    dplyr::filter(GAinS.ID %in% eigengene.patients) %>%
    unique()

In [30]:
dim(geno.fam)

[1] 638   3

In [31]:
write.table(
    geno.fam %>% dplyr::select(Family.ID, Individual.ID), 
    "~/gains_team282/nikhil/expression/eigengene_sva/mapping_patients.txt", row.names=F, quote=F, col.names=F, sep="\t"
)

Save the list of module eigengenes as a text file. This will be used by NextFlow to parallelize the association mapping.

In [32]:
write.table(colnames(eigengenes), "~/gains_team282/nikhil/expression/eigengene_sva/mapping_eigengenes.txt", row.names=F, quote=F, col.names=F, sep="\t")

### Initial Pass

I will be using a linear mixed model (LMM) to test for association between genotypes and eigengene expression. In the past, the lab has used a likelihood test (F-Test) to compare a null model of the covariates against an alternative model where the genotype is included. I will be building the following model:

1. Let $\mathbf{E}_i\in\mathbb{R}^n$ be a vector representing the values of the $i$-th eigengene.
2. Let $\mathbf{Y}\in\mathbb{R}^{n\times c}$ be a matrix of covariates. These covariates include Sex, 16 PEER factors, and 7 Genotyping PCs.
3. Let $\mathbf{Z}\in\mathbb{R}^n$ be a vector representing the random effects. The only random effect in this model is the Patient ID.
4. Let $\mathbf{X}\in\mathbb{R}^n$ be a vector representing the genotypes of the patients.
5. Let $\beta\in\mathbb{R}$ be a scalar value representing the genotypic effect on eigengene expression.
6. Let $\mathbf{\alpha}\in\mathbb{R}^c$ be a vector of covariate effects on eigengene expression.
7. Let $\gamma\in\mathbb{R}$ be a scalar value representing the random effect on eigengene expression.

The null model is:

$$\mathbf{E}_i \sim \mathbf{Y}\alpha + \mathbf{Z}\gamma$$

The alternative model is:

$$\mathbf{E}_i \sim \mathbf{X}\beta + \mathbf{Y}\alpha + \mathbf{Z}\gamma$$

There are 823 samples from the RNA-Seq data that have genotypes as well.

In [33]:
samples.with.genotypes <- rownames(eigengenes)[
    sapply(
        strsplit(rownames(eigengenes), "_"),
        function(x) { x[1] %in% geno.fam$GAinS.ID }
    )
]

length(samples.with.genotypes)

[1] 823

In [34]:
# peer.included <- peer[1:20]
# peer.included

cov.names <- c("sex", paste0("PC", 1:7))
cov.names

rand.effect.names <- c("GAinS.ID")
rand.effect.names

[1] "sex" "PC1" "PC2" "PC3" "PC4" "PC5" "PC6" "PC7"

[1] "GAinS.ID"

In [35]:
mapping.data <- merge(eigengenes, covs, by=0) %>%
    dplyr::filter(Row.names %in% samples.with.genotypes) %>%
    dplyr::select(Sample.ID=Row.names, everything()) %>%
    dplyr::mutate(GAinS.ID.NonPrefix=sapply(strsplit(Sample.ID, "_"), function(x) { x[1] })) %>%
    merge(., geno.fam, by.x="GAinS.ID.NonPrefix", by.y="GAinS.ID") %>%
    dplyr::select(GAinS.ID=Individual.ID, everything()) %>%
    dplyr::select(Sample.ID, any_of(rand.effect.names), any_of(colnames(eigengenes)), any_of(cov.names)) # , any_of(peer.included))

In [36]:
head(mapping.data)

,Sample.ID,GAinS.ID,ME_1,ME_2,ME_3,ME_4,ME_5,ME_6,ME_7,ME_8,⋯,ME_12,ME_13,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7
,<I<chr>>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,UK01050129_5,UK01050129,0.012719214,-0.023440950,-0.04542535,0.06999282,0.015125303,-0.086160570,-0.037450584,0.009992248,⋯,0.013128766,0.008309866,2,0.177279994,0.051694602,0.02486030,-0.035102502,-0.00455149,-0.0603707,0.03964880
2,UK01060123_3,UK01060123,-0.008242368,0.009466138,-0.02985941,0.00693036,-0.009508549,-0.009069338,-0.025306211,-0.021479232,⋯,-0.035444689,-0.014082065,2,-0.000388133,-0.000674099,0.00810357,0.000212791,0.03396150,0.0266317,-0.01885280
3,UK01070117_3,UK01070117,0.017011264,-0.028425084,-0.02043529,0.02811632,-0.018363903,-0.014895840,-0.005704272,-0.009856401,⋯,0.072696663,0.006468190,1,0.016432300,-0.048464298,0.00813966,-0.047699600,-0.07926830,0.0263629,-0.08224740
4,UK01080111_1,UK01080111,-0.029039958,0.031282043,-0.02566986,-0.01996656,0.011027645,0.001161878,-0.022246554,-0.035166199,⋯,0.001080440,-0.083312208,1,0.161789000,0.036224801,0.00467903,-0.048611499,0.00180036,-0.0352357,0.05012510
5,UK01110093_1,UK01110093,0.034055668,-0.040433634,0.06719405,-0.02931558,-0.047420760,0.099590607,0.079882311,-0.047126054,⋯,-0.009868418,0.036556449,2,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983
6,UK01110093_5,UK01110093,0.135468572,-0.128637208,0.08961844,0.07871120,0.007549293,-0.006789204,0.133900125,-0.015583826,⋯,0.088363899,0.074435809,2,-0.006296720,0.002821510,0.00699239,0.004351650,0.00268802,0.0166922,-0.00733983


Save the mapping data design matrix for linear mixed modeling.

In [37]:
write.csv(mapping.data, "~/gains_team282/nikhil/expression/eigengene_sva/mapping_data.csv", row.names=F)

### Identify SNPs for mQTL Mapping

Since *trans* mapping in a genome-wide fashion will require a lot of testing (the multiple-testing burden will be high), we will use SNPs that we have a more biological hypothesis for. Specifically, I will include all lead eSNPs, all lead eSNPs from the conditional cis-eQTL analysis, and related associations from the EBI GWAS Catalog.

In [38]:
length(unique(ebi.studies[,"DISEASE/TRAIT"]))

[1] 30667

After filtering based on the genotyping in GAinS, we have 124,525 studies and 56,219 SNPs.

In [39]:
# Studies with terms specified
studies <- ebi.studies[, "STUDY ACCESSION"]

# Associations from the studies that are genome-wide significant
studies.assoc <- ebi.assoc[ebi.assoc[, "STUDY ACCESSION"] %in% studies,] %>%
    dplyr::mutate(`P-VALUE` = as.numeric(`P-VALUE`)) %>%
    dplyr::filter(`P-VALUE` < 5e-8)

# Association SNPs that overlap with genotyping data in GAinS
# Ignore SNPs that are on the X chromosome
studies.assoc <- studies.assoc %>%
    dplyr::filter(SNPS %in% geno.bim$snps[geno.bim$chr != "23"])

nrow(studies.assoc)

length(unique(studies.assoc$SNPS))

[1] 124525

[1] 56219

In [40]:
mqtl.snp.table <- dplyr::bind_rows(
    lead.cis.eqtl %>%
        dplyr::mutate(source="Lead cis-eQTL SNP") %>%
        dplyr::mutate(conditional_number=NA, accession=NA) %>%
        dplyr::select(snps, source, egene=gene, conditional_number, accession),
    conditional.cis.eqtl %>%
        dplyr::mutate(source="Conditional cis-eQTL SNP") %>%
        dplyr::mutate(accession=NA) %>%
        dplyr::select(snps=SNP, source, egene=Gene, conditional_number=Number, accession),
    studies.assoc %>%
        dplyr::mutate(source="EBI GWAS Catalog") %>%
        dplyr::mutate(egene=NA, conditional_number=NA) %>%
        dplyr::select(snps=SNPS, source, egene, conditional_number, accession=37)
)

In [41]:
write.csv(mqtl.snp.table, "~/gains_team282/nikhil/expression/eigengene_sva/mqtl_snp_table.csv", quote=F, row.names=F)